### Task: Explore the Datasets and Seggregate the forwads from other players based on the position

### Import the libraries

In [20]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pymongo import MongoClient
import matplotlib.pyplot as plt

### Set up the MongoDB connection in Python

In [2]:
client=MongoClient('localhost',27017)
db=client['Wyscout']

### select only forwards from player_advance_stats and write to a DF

In [3]:
forward_details=db.player_advance_stats.aggregate([
{'$unwind': "$positions"

},
{'$project':{"_id":0,"playerId":1,"competitionId":1,"seasonId":1,"positions.position.code":1,"positions.position.name":1}
},
{'$match' : { '$or': [{"positions.position.code":"lwf"},{"positions.position.code":"rwf"},{"positions.position.code":"rw"},{"positions.position.code":"rcmf"},{"positions.position.code":"lcmf"},{"positions.position.code":"lw"},{"positions.position.code":"ss"},{"positions.position.code":"cf"},{"positions.position.code":"amf"},{"positions.position.code":"rcmf3"},{"positions.position.code":"lcmf3"},{"positions.position.code":"ramf"},{"positions.position.code":"lamf"},{"positions.position.code":"rwf"},{"positions.position.code":"lwf"}]} 
}
  
])
forward_details_df=pd.DataFrame(forward_details)

In [4]:
forward_details_df.head(5)

,playerId,competitionId,seasonId,positions
0,3319,364,185727,"{'position': {'name': 'Attacking Midfielder', ..."
1,25413,364,185727,"{'position': {'name': 'Striker', 'code': 'cf'}}"
2,25867,364,185727,"{'position': {'name': 'Striker', 'code': 'cf'}}"
3,25867,364,185727,{'position': {'name': 'Left Attacking Midfield...
4,247671,364,185727,{'position': {'name': 'Right Attacking Midfiel...


### consider only the required columns

In [5]:
forward_details_df1 = forward_details_df[['playerId','competitionId','seasonId']]

In [6]:
len(forward_details_df1)

455

### Remove duplicate records & compare the number of records

In [7]:
forward_details_df1=forward_details_df1.drop_duplicates()

In [8]:
len(forward_details_df1)

251

#### while merging getting an error: 'trying to merge on object and int64 columns' so tried converting all the relevant cols to 'int32'
#### it didn't work and hence it should be converted to csv and read it back as DF, reference:https://stackoverflow.com/questions/50649853/trying-to-merge-2-dataframes-but-get-valueerror

In [9]:
forward_details_df1.to_csv('forward_details_df1.csv')

In [10]:
forward_details_df2 = pd.read_csv('forward_details_df1.csv', index_col=0)

In [12]:
# forward_details_df1.playerId.astype(int)

#### Read the 'matches_events' into DF & as mentioned above convert it into CSV & read it back into DF

In [11]:
event_details=db.matches_events_scaling_v02.find({},{"_id":0 })
event_details_df=pd.DataFrame(event_details)
event_details_df.head(5)

,index,id,playerId,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,...,y start,x stop,y stop,tags,match_id,team_id,x start length (meters),y start length (meters),x stop length (meters),y stop length(meters)
0,0,613566210,346158,1609,2829960,1H,0.811034,8,Pass,85,...,51,28,63,1801,2829960,1609,52.5,34.68,29.4,42.84
1,1,613566221,20612,1609,2829960,1H,3.969.135,8,Pass,83,...,63,72,6,1801,2829960,1609,29.4,42.84,75.6,4.08
2,2,613566223,25867,1609,2829960,1H,7.277.111,7,Others on the ball,72,...,6,77,0,1302,2829960,1609,75.6,4.08,80.85,0
3,3,613565944,0,1625,2829960,1H,9.668.171,5,Interruption,50,...,100,,,,2829960,1609,24.15,68,0,0
4,4,613565945,8277,1625,2829960,1H,16.161.527,3,Free Kick,36,...,100,3,50,1801,2829960,1609,17.85,68,3.15,34


In [12]:
event_details_df.to_csv('event_details_df.csv')

In [13]:
event_details_df = pd.read_csv('event_details_df.csv', index_col=0)

In [14]:
event_details_df.head(2)

,index,id,playerId,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,...,y start,x stop,y stop,tags,match_id,team_id,x start length (meters),y start length (meters),x stop length (meters),y stop length(meters)
0,0,613566210,346158,1609,2829960,1H,0.811034,8,Pass,85,...,51,28.0,63.0,1801,2829960,1609,52.5,34.68,29.4,42.84
1,1,613566221,20612,1609,2829960,1H,3.969.135,8,Pass,83,...,63,72.0,6.0,1801,2829960,1609,29.4,42.84,75.6,4.08


### Merge both the dataframes on 'playerId' : Final Result

In [15]:
ForwardEventDetails=forward_details_df2.merge(event_details_df, on='playerId', how = 'left')

In [24]:
pd.set_option('display.max_columns',24)     # all the 24 columns will be displayed
ForwardEventDetails.head(3)

,playerId,competitionId,seasonId,index,id,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,subEventName,x start,y start,x stop,y stop,tags,match_id,team_id,x start length (meters),y start length (meters),x stop length (meters),y stop length(meters)
0,3319,364,185727,112.0,480360228.0,1609.0,2829966.0,1H,402.088.725,7.0,Others on the ball,72.0,Touch,55.0,89.0,67.0,72.0,0,2829966.0,1609.0,57.75,60.52,70.35,48.96
1,3319,364,185727,113.0,480360229.0,1609.0,2829966.0,1H,408.741.947,8.0,Pass,85.0,Simple pass,67.0,72.0,60.0,37.0,1801,2829966.0,1609.0,70.35,48.96,63.00,25.16
2,3319,364,185727,119.0,480360235.0,1609.0,2829966.0,1H,42.589.196,8.0,Pass,85.0,Simple pass,75.0,79.0,69.0,56.0,1801,2829966.0,1609.0,78.75,53.72,72.45,38.08


In [25]:
ForwardEventDetails.isnull().values.any()   # There are NO null values, as they were replaced with 0 in the next step
ForwardEventDetails.isnull().sum().sum()

0

In [26]:
ForwardEventDetails=ForwardEventDetails.replace(np.nan,0)  # replace 'NaN with 0'
ForwardEventDetails.head(5)

,playerId,competitionId,seasonId,index,id,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,subEventName,x start,y start,x stop,y stop,tags,match_id,team_id,x start length (meters),y start length (meters),x stop length (meters),y stop length(meters)
0,3319,364,185727,112.0,480360228.0,1609.0,2829966.0,1H,402.088.725,7.0,Others on the ball,72.0,Touch,55.0,89.0,67.0,72.0,0,2829966.0,1609.0,57.75,60.52,70.35,48.96
1,3319,364,185727,113.0,480360229.0,1609.0,2829966.0,1H,408.741.947,8.0,Pass,85.0,Simple pass,67.0,72.0,60.0,37.0,1801,2829966.0,1609.0,70.35,48.96,63.00,25.16
2,3319,364,185727,119.0,480360235.0,1609.0,2829966.0,1H,42.589.196,8.0,Pass,85.0,Simple pass,75.0,79.0,69.0,56.0,1801,2829966.0,1609.0,78.75,53.72,72.45,38.08
3,3319,364,185727,188.0,480360312.0,1609.0,2829966.0,1H,638.084.147,8.0,Pass,85.0,Simple pass,43.0,51.0,46.0,86.0,1801,2829966.0,1609.0,45.15,34.68,48.30,58.48
4,3319,364,185727,191.0,480360315.0,1609.0,2829966.0,1H,645.601.945,8.0,Pass,85.0,Simple pass,62.0,75.0,68.0,62.0,1801,2829966.0,1609.0,65.10,51.00,71.40,42.16
